In [6]:
import pandas as pd
import numpy as np
from loaders import *

In [12]:
vgg_layers = [
#     ConfigRegistry.VGG_LAYER1,
#     ConfigRegistry.VGG_LAYER2,
    ConfigRegistry.VGG_LAYER3,
#     ConfigRegistry.VGG_LAYER4,
    ConfigRegistry.VGG_LAYER5,
#     ConfigRegistry.VGG_LAYER6,
#     ConfigRegistry.VGG_LAYER7,
#     ConfigRegistry.VGG_LAYER8,
#     ConfigRegistry.VGG_LAYER9,
#     ConfigRegistry.VGG_LAYER10,
#     ConfigRegistry.VGG_LAYER11,
#     ConfigRegistry.VGG_LAYER12,
#     ConfigRegistry.VGG_LAYER13,
]

vgg_maps = [
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L1,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L2,
    ConfigRegistry.ALBIREO_MAPPING_VGG_L3,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L4,
    ConfigRegistry.ALBIREO_MAPPING_VGG_L5,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L6,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L7,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L8,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L9,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L10,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L11,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L12,
#     ConfigRegistry.ALBIREO_MAPPING_VGG_L13,
]

In [3]:
!accelergyTables -r /home/workspace/final-project/example_designs/albireo/albireo_estimation_tables

albireo_accelergy_result = run_accelergy(
    ConfigRegistry.ALBIREO_ARCH,
    ConfigRegistry.ALBIREO_COMPONENTS_DIR
)
print(albireo_accelergy_result.ert_verbose)

/home/workspace/final-project/example_designs/albireo/albireo_estimation_tables is already added as a root for table based plug-in
ERT_summary:
  version: 0.3
  table_summary:
  - name: system_arch.chip.plcg[0..8].plcu[0..2].psumBuffer[0..4]
    actions:
    - name: read
      average_energy: 1.0
      max_energy: 1
      min_energy: 1
    - name: write
      average_energy: 1.0
      max_energy: 1
      min_energy: 1
    - name: idle
      energy: 0
    primitive_estimation(s):
    - system_arch.chip.plcg[0..8].plcu[0..2].psumBuffer[0..4]:
        estimator: dummy_table
  - name: system_arch.DRAM
    actions:
    - name: read
      energy: 512
    - name: write
      energy: 512
    - name: idle
      energy: 0
    primitive_estimation(s):
    - system_arch.DRAM:
        estimator: Cacti
  - name: system_arch.chip.plcg[0..8].plcu[0..2].plcu_dummy_buffer
    actions:
    - name: access
      energy: 0
    primitive_estimation(s):
    - action_name: access
      arguments: null
      en

In [4]:
for i, layer in enumerate(vgg_layers):
    results = run_timeloop_model(
        ConfigRegistry.ALBIREO_ARCH,
        ConfigRegistry.ALBIREO_COMPONENTS_DIR,
        vgg_maps[i],
        layer
    )
    print(f"Layer: {i+1}")
    stats, loops = results
    print(loops)

input file: tmp.yaml
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-model. -o ./ > timeloop-model.accelergy.log 2>&1


Utilization = 1.00 | pJ/Compute =    4.591
Layer: 1

DRAM [ Inputs:153228 (153228) Outputs:3211264 (3211264) ] 
---------------------------------------------------------
| for Q in [0:45)

shared_glb [ Weights:1728 (1728) Inputs:4746 (4746) Outputs:71680 (71680) ] 
---------------------------------------------------------------------------
|   for M in [0:8)
|     for P in [0:224)

dummy_buffer [ ] 
----------------
|       for Q in [0:1)
|         for M in [0:9,1) (Spatial-X)

weight_cache [ Weights:27 (27) ] 
--------------------------------
|           for Q in [0:1)
|             for C in [0:3) (Spatial-X)

plcu_dummy_buffer [ ] 
---------------------
|               for Q in [0:1)
|                 for Q in [0:5,4) (Spatial-X)

psumBuffer [ Outputs:1 (1) ] 
----------------------------
|                   for S in [0:3)
|                     for R in [0:3)


input file: tmp.yaml
execute:/usr/local/bin/accelergy tmp.yaml --oprefix timeloop-model. -o ./ > timeloop-model.accelergy.lo

KeyboardInterrupt: 

In [14]:
vgg_albireo_layerwise_results = []

for i, layer in enumerate(vgg_layers):
    albireo_results = run_timeloop_mapper(
        ConfigRegistry.ALBIREO_ARCH,
        ConfigRegistry.ALBIREO_COMPONENTS_DIR,
        albireo_accelergy_result.art,
        albireo_accelergy_result.ert,
        ConfigRegistry.ALBIREO_CONSTRAINTS_VGG,
        ConfigRegistry.ALBIREO_MAPPER,
        layer,
    )
    print(f"Layer: {i+1}")
    albireo_results_stats, albireo_results_loops = albireo_results
#     print(albireo_results_loops)
#     print(albireo_results_stats)
    vgg_albireo_layerwise_results.append(albireo_results)

input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
Found Accelergy ERT (energy reference table), replacing internal energy model.
Found Accelergy ART (area reference table), replacing internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 336
  Factorization options along problem dimension M = 792
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization options along problem dimension N = 1
  Factorization options along problem dimension P = 756
  Factorization options a

[  4] Utilization = 0.05 | pJ/Compute =    4.320 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 C4 - L1[] P2 M2 C4 - L0[O] S3 R3 
[  0] Utilization = 0.05 | pJ/Compute =    4.285 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 - L1[] P2 M2 C16 - L0[O] S3 R3 
[  3] Utilization = 0.10 | pJ/Compute =    4.309 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 C2 C2X - L1[] P2 M2 C4 - L0[O] S3 R3 
[  6] Utilization = 0.05 | pJ/Compute =    4.462 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 C16 - L1[] P2 M2 - L0[O] S3 R3 
[  1] Utilization = 0.10 | pJ/Compute =    4.290 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 C2X - L1[] P2 M2 C8 - L0[O] S3 R3 
[  5] Utilization = 0.10 | pJ/Compute =    4.346 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] Q14 M4 C4 C2X - L1[] P2 M2 C2 - L0[O] S3 R3 
[  2] Utilization = 0.05 | pJ/Compute =    4.297 | L5[IO] Q8 P2 M2 C2 - L4[IO] M8 - L3[] P4 C2 P7X - L2[W] 



Summary stats for best mapping found by mapper:
  Utilization = 0.10 | pJ/Compute =    4.290
Layer: 1
input file: tmp.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
Found Accelergy ERT (energy reference table), replacing internal energy model.
Found Accelergy ART (area reference table), replacing internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dimension C = 540
  Factorization options along problem dimension M = 1287
  Factorization options along problem dimension R = 1
  Factorization options along problem dimension S = 1
  Factorization options along 

[  1] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  2] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  3] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  7] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  5] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  4] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  6] STATEMENT: 1500 invalid mappings (1499 fanout, 1 capacity) found since the last valid mapping, terminating search.
[  0] Utilization = 0.03 | pJ/Compute =    3.242 | L5[IO] Q2 M2 C16 - L4[IO] P2 M4 - L3[] Q7 C4 M4X - L2[W] P4 M2 C2 - L1[] Q4 P7 M4 - L0[O] S3 R3 
[  0]



Summary stats for best mapping found by mapper:
  Utilization = 0.03 | pJ/Compute =    3.242
Layer: 2


In [9]:
for i, (stats, loops) in enumerate(vgg_albireo_layerwise_results):
    print(f"Layer: {i+1}")
    log = False
    for line in stats.split("\n"):
        if "Summary Stats" in line:
            log = True
        if log:
            print(line)

Layer: 1
Summary Stats
-------------
GFLOPs (@1GHz): 7.85
Utilization: 0.03
Cycles: 21676032
Energy: 484.32 uJ
EDP(J*cycle): 1.05e+04
Area: 0.00 mm^2

Computes = 86704128
pJ/Compute
    omacs                               = 0.07
    psumBuffer                          = 2.07
    plcu_dummy_buffer                   = 0.00
    weight_cache                        = 0.50
    dummy_buffer                        = 0.00
    shared_glb                          = 0.12
    DRAM                                = 2.50
    A2D_NoC                             = 0.02
    D2A_NoC                             = 0.30
    DRAM <==> shared_glb                = 0.00
    dummy_buffer <==> weight_cache      = 0.00
    plcu_dummy_buffer <==> psumBuffer   = 0.00
    psumBuffer <==> omacs               = 0.00
    weight_cache <==> plcu_dummy_buffer = 0.00
    Total                               = 5.59


Layer: 2
Summary Stats
-------------
GFLOPs (@1GHz): 31.97
Utilization: 0.12
Cycles: 28901376
Energy: 1363.90 u

In [10]:
total_cycles = 0
for i, (stats, loops) in enumerate(vgg_albireo_layerwise_results):
    for line in stats.split("\n"):
        if "Cycles: " in line:
            cycles = int(line[len("Cycles: "):])
            total_cycles += cycles
            print(f"Cycles in layer {i+1}: {cycles}")
            break
print(f"Albireo total cycles: {total_cycles}")

Cycles in layer 1: 21676032
Cycles in layer 2: 28901376
Cycles in layer 4: 115605504
Cycles in layer 6: 231211008
Cycles in layer 7: 231211008
Cycles in layer 8: 231211008
Cycles in layer 9: 231211008
Cycles in layer 10: 28901376
Cycles in layer 11: 28901376
Cycles in layer 12: 28901376
Cycles in layer 13: 28901376
Albireo total cycles: 1206632448


In [11]:
total_energy = 0
for i, (stats, loops) in enumerate(vgg_albireo_layerwise_results):
    for line in stats.split("\n"):
        if "Energy: " in line:
            energy = float(line[len("Energy: "):].split(' uJ')[0])
            total_energy += energy
            print(f"Energy in layer {i+1}: {energy}")
            break
print(f"Total energy: {total_energy} uJ")

Energy in layer 1: 484.32
Energy in layer 2: 1363.9
Energy in layer 4: 1525.6
Energy in layer 6: 6051.39
Energy in layer 7: 3017.49
Energy in layer 8: 7473.71
Energy in layer 9: 7473.71
Energy in layer 10: 1333.49
Energy in layer 11: 1333.49
Energy in layer 12: 1333.49
Energy in layer 13: 1333.49
Total energy: 32724.080000000005 uJ
